In [1]:
import pandas as pd
import networkx as nx

In [2]:
from iac_sketch import data, etl, sketch, transform

In [3]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(etl)
importlib.reload(transform)
importlib.reload(sketch)
# architect = sketch.Architect("../public/components")
# registry = architect.parse()

<module 'iac_sketch.sketch' from '/Users/zhafen/repos/iac-sketch/iac_sketch/sketch.py'>

In [31]:
from pandera.engines import pandas_engine


In [34]:
import pandera as pa

In [4]:
extract_sys = etl.ExtractSystem()
registry = extract_sys.extract_entities("../public/components/*yaml")
registry

In [5]:
transform_sys = etl.TransformSystem()
# Normalize components in the registry
excluded_components = ["fields"]
registry = transform_sys.apply_transform(
    registry,
    transform.ComponentNormalizer(),
    components_mapping={comp: data.View(comp) for comp in registry.components if comp not in excluded_components},
)

In [6]:
# We do an outer join here because we count "fields" as an implicit flag for a component.
X = registry.view(["component", "fields"], join_on="entity", join_how="outer")
X

,entity,comp_ind,multiplicity,flag,comp_ind.fields,component
0,alternative,0.0,NaN,NaN,NaN,NaN
1,case,0.0,NaN,NaN,2.0,"{'case [dict[str, str]]': 'Case statement. Can..."
2,code_location_format,NaN,NaN,NaN,1.0,{'code_type [str]': 'What type of code this is...
3,cohort,NaN,NaN,NaN,1.0,"{'config_id [str]': 'Unique ID for a cohort.',..."
4,cohort_patient,NaN,NaN,NaN,1.0,"{'patient [entity]': None, 'cohort [entity]': ..."
5,cohort_patients,NaN,NaN,NaN,2.0,"{'cohort [entity]': None, 'patient [entity]': ..."
6,cohort_patients_history,NaN,NaN,NaN,2.0,"{'cohort [entity]': None, 'patient [entity]': ..."
7,component,0.0,1,NaN,2.0,{'multiplicity [str]': {'description': 'Allowe...
8,databricks_workspace,NaN,NaN,NaN,1.0,"{'url [str]': 'Unique URL for the workspace.',..."
9,dependency,0.0,NaN,NaN,NaN,NaN


In [7]:
# Add in all components defined in the registry
# and mark the ones that are not defined
X["defined"] = True
registry_comps = pd.DataFrame({"entity": registry.keys()})
X = X.merge(registry_comps, how="outer", on="entity")
X.loc[X["defined"].isna(), "defined"] = False
X["defined"] = X["defined"].astype(bool)
X

,entity,comp_ind,multiplicity,flag,comp_ind.fields,component,defined
0,algorithm,NaN,NaN,NaN,NaN,NaN,False
1,alternative,0.0,NaN,NaN,NaN,NaN,True
2,alternative_to,NaN,NaN,NaN,NaN,NaN,False
3,append_to,NaN,NaN,NaN,NaN,NaN,False
4,associated_components,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...
56,url,NaN,NaN,NaN,NaN,NaN,False
57,used_in,NaN,NaN,NaN,NaN,NaN,False
58,website,NaN,NaN,NaN,NaN,NaN,False
59,work_item,NaN,NaN,NaN,1.0,"{'azdo_id [int]': 'The ID provided by AzDO.', ...",True


In [ ]:
def parse_fields(row):
    if pd.isna(row["component"]):
        # Flags are components without any fields. Think of them like tags.
        if row["flag"]:
            row["valid"] = True
            row["errors"] = ""
        else:
            row["valid"] = False
            row["errors"] = "Fields definition is missing."
        return row

    fields_i = {}
    valid_fields = True
    valid_message = ""
    for field_key, field_value in row["component"].items():
        try:
            field = data.Field.from_kv_pair(field_key, field_value)
            fields_i[field.name] = field
        except ValueError:
            valid_fields = False
            valid_message = (
                f"Field {field_key} is incorrectly formatted: {field_value}. "
            )
            break

    row["fields"] = fields_i
    row["valid"] = valid_fields
    row["errors"] = valid_message

    return row

In [ ]:
X.apply(parse_fields, axis="columns")

In [41]:

from pandera import dtypes

dt = dtypes.DataType("int")

TypeError: DataType.__init__() takes 1 positional argument but 2 were given

In [40]:
dtypes.DataType.__dir__()

TypeError: unbound method object.__dir__() needs an argument

In [38]:
dtypes.String().check(pandas_engine.Engine.dtype("str")) 

False

In [ ]:
# Create a directed graph from the DataFrame
graph = nx.from_pandas_edgelist(
    registry["link"],
    source="source",
    target="target",
    edge_key="link_type",
    create_using=nx.DiGraph,
)

# Visualize the graph
nx.draw(graph, with_labels=True)

In [ ]:
graph.add_nodes_from(registry["metadata"]["entity"].values)

In [ ]:
connected_components = [_ for _ in nx.connected_components(graph.to_undirected())]
metadata = registry["metadata"].set_index("entity")
metadata["connected_component_category"] = -1
for i, comps in enumerate(connected_components):
    metadata.loc[list(comps), "connected_component_category"] = i

In [ ]:
pair_connectivity = nx.all_pairs_node_connectivity(graph)

In [ ]:
sum(pair_connectivity["accept_request"].values()) > 0

In [ ]:
# Find invalid requirements
reqs = registry.view(["requirement", "status", "task", "input"])
is_valid = reqs["status"].isin(["closed", "removed"])
invalid_reqs = reqs.loc[~is_valid]
invalid_reqs

In [ ]:
# Find invalid testcases
registry.view(["testcase", "status"])

In [ ]:
import base64
import io, requests
from IPython.display import Image, display
from PIL import Image as im
import matplotlib.pyplot as plt

In [ ]:


def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    img = im.open(io.BytesIO(requests.get('https://mermaid.ink/img/' + base64_string).content))
    plt.imshow(img)
    plt.axis('off') # allow to hide axis

In [ ]:
graph = """
graph LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A
    E--> B & C & D
"""

In [ ]:
graphbytes = graph.encode("utf8")
base64_bytes = base64.urlsafe_b64encode(graphbytes)
base64_string = base64_bytes.decode("ascii")
result = requests.get('https://mermaid.ink/img/' + base64_string).content
parsed_result = io.BytesIO(result)
img = im.open(parsed_result)
img

In [ ]:
mm("""
flowchart LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A & E
    E--> B & C & D
""")